## Import Packages

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [2]:
data = pd.read_csv('data.csv')
data.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Time_updated,Year,Month
0,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,2012-09-06 17:00:00,2012,9
1,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,2012-10-20 17:00:00,2012,10
2,5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,2012-07-11 17:00:00,2012,7
3,6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,2012-06-19 17:00:00,2012,6
4,7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,2012-05-02 17:00:00,2012,5


In [3]:
data.shape

(198064, 14)

## Sentimental Analysis

### Calculate sentimental score for each review

In [16]:
# Import sentimental analysis package
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Get the value from the the "Text" column
Tweets = data[['Text']].values.tolist()
# Get the compound sentimental score for each review and append to a list
analyzer = SentimentIntensityAnalyzer()
Sentiment = []
for line in Tweets:
    Sentiment.append(analyzer.polarity_scores(line[0])['compound'])
# Add a column to data frame for the sentimental score
data['Sentiment'] = Sentiment

In [24]:
data.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Time_updated,Year,Month,Sentiment
0,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,2012-09-06 17:00:00,2012,9,-0.5664
1,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,2012-10-20 17:00:00,2012,10,0.9468
2,5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,2012-07-11 17:00:00,2012,7,0.8830
3,6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,2012-06-19 17:00:00,2012,6,0.9346
4,7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,2012-05-02 17:00:00,2012,5,0.9487


### Good Product 

In [27]:
# Take the average sentiment score for each product as their final score
data_high=data.groupby('ProductId').agg({"Sentiment":"mean"})\
                    .sort_values('Sentiment',ascending = False).reset_index()

In [28]:
data_high.head(10)

,ProductId,Sentiment
0,B0001H1RO0,0.9997
1,B004R6F88W,0.9995
2,B001B4NFXI,0.9995
3,B007B24NKS,0.9994
4,B008ADQO1I,0.9991
5,B008ADQNXW,0.9991
6,B008ADQNZA,0.9991
7,B005HQQSAI,0.9989
8,B004D4EII4,0.9989
9,B001VE2Q2E,0.9987


In [35]:
# Choose the product with the highest score to see the details of the review
data_high_detail=data[(data["ProductId"]=="B0001H1RO0")]

In [38]:
data_high_detail[['Text']].values

array([['And we find her here upon this grand amazon!<br /><br />Each day Morinu feeds me like my own nourishing mother, cherishes and consents my whims with this wonderful food.<br /><br />Eschew all animal byproducts and cadavers.<br /><br />This tofu is a strengthening food, fit for human consumption.<br /><br />And so convenient from our mighty amazon, sent right to your front porch or, in my case, post office box with free shipping rather than driving thirty to seventy five miles for the same product at three times the price, plus gas.<br /><br />This is fit food for humans, food for which our bodies are created, food which feeds us rather than kills both the source and the consumer.<br /><br />You may read here written by the self-deceived that this tofu is too mushy for firmness and that it must be pressed between towels before cooking. To them so confused I ask<br /><br />Cooking?<br /><br />Each day at least once a day, I draw forth my serrated knife, and one sacrificial box o

In [29]:
# Select top 10 product with the highest score as good product
good_prod=data_high.head(10)

In [30]:
# Write the good product files into CSV
good_prod.to_csv("good_prod.csv")

### Bad Product 

In [31]:
# Take the average sentiment score for each product as their final score
data_low=data.groupby('ProductId').agg({"Sentiment":"mean"})\
                    .sort_values('Sentiment',ascending = True).reset_index()

In [32]:
data_low.head(10)

,ProductId,Sentiment
0,B0025UNRLW,-0.9957
1,B000WFLQR4,-0.9918
2,B000WFEODC,-0.9892
3,B00008GQ50,-0.9892
4,B002X9HAV8,-0.9887
5,B003M58692,-0.9863
6,B006Z46G3K,-0.9846
7,B0025UC8JO,-0.9835
8,B0025ULWVE,-0.9820
9,B000GPY38E,-0.9803


In [44]:
# Choose the product with the lowest score to see the details of the review
data_low_detail=data[(data["ProductId"]=="B0025UNRLW")]

In [45]:
data_low_detail[['Text']].values

array([['Since this is individually packaged sweet pickle relish why on earth is FD&C Blue 1 (which is derived from petroleum) included as an ingredient?<br /><br />It is bad enough these pickle relish packets contain HFCS, or HIGH FRUCTOSE CORN SYRUP which NONE of us need for our ever expanding mid-lines, but it also contains FD&C BLUE 1 of which some of us cannot consume without dire consequences!  FD&C BLUE 1 causes severe abdominal distress in a percentage of the population (many of whom have yet to make the connection).  If you aren\'t sure why Blue 1 is such a bad idea, check out the FDA press release stating blue 1 is "casually linked" to fatalities they investigated.<br /><br />I recently attended an event where these relish packets (and another brand - also contains blue 1) were served. Stoopidly I dumped the packet onto my food and a few hours later, like clockwork, I was in severe abdominal distress. Major GI issues. Checking around, sure enough, these packets show blue 1 in

In [33]:
#Write the bad product files into CSV
bad_prod=data_low.head(10)
bad_prod.to_csv("bad_prod.csv")